In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd
from functools import reduce

In [88]:
england_std21 = pd.read_excel('data/league_data/england/21/england_std.xlsx')

france_std21 = pd.read_excel('data/league_data/france/21/france_std.xlsx')
france_std21.columns = england_std21.columns

germany_std21 = pd.read_excel('data/league_data/germany/21/germany_std.xlsx')
germany_std21.columns = england_std21.columns

italy_std21 = pd.read_excel('data/league_data/italy/21/italy_std.xlsx')
italy_std21.columns = england_std21.columns

spain_std21 = pd.read_excel('data/league_data/spain/21/spain_std.xlsx')
spain_std21.columns = england_std21.columns



s21_std = pd.concat([england_std21,france_std21,germany_std21,italy_std21,spain_std21])

In [83]:
def clean_std(dataframe):

    original = []
    duplicate = []

    for a in dataframe['Player']:
        if a not in original:
            original.append(a)
        else:
            duplicate.append(a)
    
    mid_season_transfers = pd.DataFrame(columns = dataframe.columns)

    for a in duplicate:
        player = dataframe[dataframe['Player']==a]
        mid_season_transfers = mid_season_transfers.append(dataframe[dataframe['Player']==a])
    
    
    first = mid_season_transfers.iloc[::2]
    second = mid_season_transfers.iloc[1::2]
    
    midseason = []

    for a in list(range(len(first))):

        double_club_combined = pd.Series([first.iloc[a]['Player'],
                           [first.iloc[a]['Squad'],second.iloc[a]['Squad']],
                           first.iloc[a]['MP']+second.iloc[a]['MP'],
                            first.iloc[a]['Starts']+second.iloc[a]['Starts'],
                            first.iloc[a]['Min']+second.iloc[a]['Min'],
                           (first.iloc[a]['Min']+second.iloc[a]['Min'])/90,
                           first.iloc[a]['Gls']+second.iloc[a]['Gls'],
                           first.iloc[a]['Ast']+second.iloc[a]['Ast'],
                           first.iloc[a]['Non-Penalty Goals']+second.iloc[a]['Non-Penalty Goals'],
                           first.iloc[a]['Penalties Scored']+second.iloc[a]['Penalties Scored'],
                           first.iloc[a]['Penalties Attempted']+second.iloc[a]['Penalties Attempted'],
                           first.iloc[a]['Yellow Cards']+second.iloc[a]['Yellow Cards'],
                           first.iloc[a]['Red Cards']+second.iloc[a]['Red Cards'],
                           (first.iloc[a]['Gls']+second.iloc[a]['Gls'])/90,
                           (first.iloc[a]['Ast']+second.iloc[a]['Ast'])/90,
                           ((first.iloc[a]['Gls']+second.iloc[a]['Gls'])+(first.iloc[a]['Ast']+second.iloc[a]['Ast']))/90,
                           ((first.iloc[a]['Gls']+second.iloc[a]['Gls'])-(first.iloc[a]['Penalties Scored']+second.iloc[a]['Penalties Scored']))/90,
                           ((first.iloc[a]['Gls']+second.iloc[a]['Gls'])+(first.iloc[a]['Ast']+second.iloc[a]['Ast'])-(first.iloc[a]['Penalties Scored']+second.iloc[a]['Penalties Scored']))/90,
                           first.iloc[a]['xG']+second.iloc[a]['xG'],
                           first.iloc[a]['Non-Penalty xG']+second.iloc[a]['Non-Penalty xG'],
                           first.iloc[a]['xA']+second.iloc[a]['xA'],
                           first.iloc[a]['Non-penalty xG+ xA']+second.iloc[a]['Non-penalty xG+ xA'],
                           (first.iloc[a]['xG']+second.iloc[a]['xG'])/90,
                           (first.iloc[a]['xA']+second.iloc[a]['xA'])/90,
                           ((first.iloc[a]['xG']+second.iloc[a]['xG'])+(first.iloc[a]['xA']+second.iloc[a]['xA']))/90,
                           (first.iloc[a]['Non-Penalty xG']+second.iloc[a]['Non-Penalty xG'])/90,
                           (first.iloc[a]['Non-penalty xG+ xA']+second.iloc[a]['Non-penalty xG+ xA'])/90])

        midseason.append(double_club_combined)

    transferred_players = pd.DataFrame(midseason)
    transferred_players.columns = ['Player','Squad','MP','Starts','Min',
                                   'Total Mins/90','Gls','Ast','Non-Penalty Goals',
                                    'Penalties Scored','Penalties Attempted','Yellow Cards',
                                   'Red Cards','Gls/90','Ast/90','(G+A)/90','Non-Penalty Goals/90',
                                   '(Gls+Ast-Scored Penalties)/90','xG','Non-Penalty xG','xA',
                                    'Non-penalty xG+ xA','xG/90','xA/90','(xG+xA)/90',
                                     'Non-Penalty xG/90','(Non-Penalty xG+xA)/90']
    
    final_df = pd.concat([dataframe, mid_season_transfers, mid_season_transfers]).drop_duplicates(keep=False)
    final_df = pd.concat([final_df,transferred_players])
    return final_df
    

In [84]:
clean_std(england_std21)

,Player,Squad,MP,Starts,Min,Total Mins/90,Gls,Ast,Non-Penalty Goals,Penalties Scored,...,(Gls+Ast-Scored Penalties)/90,xG,Non-Penalty xG,xA,Non-penalty xG+ xA,xG/90,xA/90,(xG+xA)/90,Non-Penalty xG/90,(Non-Penalty xG+xA)/90
0,Pierre Højbjerg,Tottenham,38,38,3420,38.000000,2,4,2,0,...,0.160000,1.1,1.1,1.9,3.0,0.030000,0.050000,0.080000,0.030000,0.080000
1,Hugo Lloris,Tottenham,38,38,3420,38.000000,0,0,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,Emiliano Martínez,Aston Villa,38,38,3420,38.000000,0,0,0,0,...,0.000000,0.0,0.0,0.2,0.2,0.000000,0.010000,0.010000,0.000000,0.010000
3,Aaron Ramsdale,Sheffield Utd,38,38,3420,38.000000,0,0,0,0,...,0.000000,0.0,0.0,0.2,0.2,0.000000,0.000000,0.000000,0.000000,0.000000
4,Kasper Schmeichel,Leicester City,38,38,3420,38.000000,0,0,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,Takumi Minamino,"[Southampton, Liverpool]",19,11,1004,11.155556,3,0,3,0,...,0.033333,1.9,1.9,0.5,2.4,0.021111,0.005556,0.026667,0.021111,0.026667
2,Mathew Ryan,"[Brighton, Arsenal]",14,14,1260,14.000000,0,0,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,Joe Willock,"[Newcastle Utd, Arsenal]",21,13,1216,13.511111,8,0,8,0,...,0.088889,4.0,3.0,0.7,3.9,0.044444,0.007778,0.052222,0.033333,0.043333
4,Ruben Loftus-Cheek,"[Fulham, Chelsea]",31,22,1969,21.877778,1,0,1,0,...,0.011111,3.1,3.1,1.4,4.5,0.034444,0.015556,0.050000,0.034444,0.050000
